# MC Dropout with Fire Detection

## **1. Fire Detection Dataset 다운로드**

In [10]:
import os

os.system("kaggle datasets download atulyakumar98/test-dataset")
os.makedirs("data", exist_ok = True)
os.system("unzip test-dataset.zip -d ./data")
os.system("rm test-dataset.zip")

Dataset URL: https://www.kaggle.com/datasets/atulyakumar98/test-dataset
License(s): unknown


100%|██████████| 132M/132M [00:05<00:00, 25.6MB/s] 



Archive:  test-dataset.zip
  inflating: ./data/Fire-Detection/0/1.jpg  
  inflating: ./data/Fire-Detection/0/10.jpg  
  inflating: ./data/Fire-Detection/0/100.jpg  
  inflating: ./data/Fire-Detection/0/101.jpg  
  inflating: ./data/Fire-Detection/0/102.jpg  
  inflating: ./data/Fire-Detection/0/103.jpg  
  inflating: ./data/Fire-Detection/0/104.jpg  
  inflating: ./data/Fire-Detection/0/105.jpg  
  inflating: ./data/Fire-Detection/0/106.jpg  
  inflating: ./data/Fire-Detection/0/107.jpg  
  inflating: ./data/Fire-Detection/0/108.jpg  
  inflating: ./data/Fire-Detection/0/109.jpg  
  inflating: ./data/Fire-Detection/0/11.jpeg  
  inflating: ./data/Fire-Detection/0/110.jpg  
  inflating: ./data/Fire-Detection/0/111.jpg  
  inflating: ./data/Fire-Detection/0/112.jpg  
  inflating: ./data/Fire-Detection/0/113.jpg  
  inflating: ./data/Fire-Detection/0/114.jpg  
  inflating: ./data/Fire-Detection/0/115.jpg  
  inflating: ./data/Fire-Detection/0/116.jpg  
  inflating: ./data/Fire-Detection/

0

## **2. 기초 통계량 확인**

In [11]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from collections import Counter
import numpy as np
import os
from PIL import Image
import random

import torch
import torchvision
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Dataset, Subset
import torch.nn as nn
import torch.optim as optim

from sklearn.model_selection import train_test_split

In [12]:
def set_seed(seed_value=42):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

seed_value = 42
set_seed(seed_value)

In [ ]:
class CustomImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = sorted(os.listdir(root_dir))
        self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)}
        self.image_paths = []
        self.labels = []

        for cls_name in self.classes:
            cls_path = os.path.join(root_dir, cls_name)
            for img_name in os.listdir(cls_path):
                img_path = os.path.join(cls_path, img_name)
                self.image_paths.append(img_path)
                self.labels.append(self.class_to_idx[cls_name])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert('RGB')  

        if self.transform:
            image = self.transform(image)

        return image, label

data_dir = 'data/Fire-Detection'

image_size = 224

## ImageNet mean / std: 변경 필요
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

full_dataset = CustomImageDataset(root_dir=data_dir, transform=transform)

labels = [label for _, label in full_dataset]
class_counts = Counter(labels)
print('Class Distribution:', class_counts)

Class Distribution: Counter({0: 541, 1: 110})


* 데이터셋 분할 후 train set에서 mean/std 구해서 정규화해야 할 듯

## MC dropout

In [ ]:
# Source - https://stackoverflow.com/a
# Posted by Iswariya Manivannan, modified by community. See post 'Timeline' for change history
# Retrieved 2025-12-09, License - CC BY-SA 4.0

import sys

import numpy as np

import torch
import torch.nn as nn

def enable_dropout(model):
    """ Function to enable the dropout layers during test-time """
    for m in model.modules():
        if m.__class__.__name__.startswith('Dropout'):
            m.train()

def get_monte_carlo_predictions(data_loader,
                                forward_passes,
                                model,
                                n_classes,
                                n_samples):
    """ Function to get the monte-carlo samples and uncertainty estimates
    through multiple forward passes

    Parameters
    ----------
    data_loader : object
        data loader object from the data loader module
    forward_passes : int
        number of monte-carlo samples/forward passes
    model : object
        keras model
    n_classes : int
        number of classes in the dataset
    n_samples : int
        number of samples in the test set
    """

    dropout_predictions = np.empty((0, n_samples, n_classes))
    softmax = nn.Softmax(dim=1)
    for i in range(forward_passes):
        predictions = np.empty((0, n_classes))
        model.eval()
        enable_dropout(model)
        for i, (image, label) in enumerate(data_loader):
            image = image.to(torch.device('cuda'))
            with torch.no_grad():
                output = model(image)
                output = softmax(output)  # shape (n_samples, n_classes)
            predictions = np.vstack((predictions, output.cpu().numpy()))

        dropout_predictions = np.vstack((dropout_predictions,
                                         predictions[np.newaxis, :, :]))
        # dropout predictions - shape (forward_passes, n_samples, n_classes)

    # Calculating mean across multiple MCD forward passes 
    mean = np.mean(dropout_predictions, axis=0)  # shape (n_samples, n_classes)

    # Calculating variance across multiple MCD forward passes 
    variance = np.var(dropout_predictions, axis=0)  # shape (n_samples, n_classes)

    epsilon = sys.float_info.min
    # Calculating entropy across multiple MCD forward passes 
    entropy = -np.sum(mean * np.log(mean + epsilon), axis=-1)  # shape (n_samples,)

    # Calculating mutual information across multiple MCD forward passes 
    mutual_info = entropy - np.mean(np.sum(-dropout_predictions * np.log(dropout_predictions + epsilon),
                                           axis=-1), axis=0)  # shape (n_samples,)
